**This notebook is an exercise in the [Advanced SQL](https://www.kaggle.com/learn/advanced-sql) course.  You can reference the tutorial at [this link](https://www.kaggle.com/alexisbcook/analytic-functions).**

---


# Introduction

Here, you'll use window functions to answer questions about the [Chicago Taxi Trips](https://www.kaggle.com/chicago/chicago-taxi-trips-bq) dataset.

Before you get started, run the code cell below to set everything up.

In [1]:
# Set up feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.sql_advanced.ex2 import *
print("Setup Complete")

Using Kaggle's public dataset BigQuery integration.


/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


Setup Complete


The following code cell fetches the `taxi_trips` table from the `chicago_taxi_trips` dataset. We also preview the first five rows of the table.  You'll use the table to answer the questions below.

In [2]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "chicago_taxi_trips" dataset
dataset_ref = client.dataset("chicago_taxi_trips", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "taxi_trips" table
table_ref = dataset_ref.table("taxi_trips")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()

Using Kaggle's public dataset BigQuery integration.


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the tabledata.list endpoint.


,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location
0,f0d00207c546d46b70e45593e08c3bdde2a621f1,6db76715006643fd2c01bce95f1ccb0e4898c982bdd49e...,2015-03-23 07:45:00+00:00,2015-03-23 08:00:00+00:00,1020,4.8,NaN,NaN,NaN,NaN,...,0.0,13.45,Cash,None,NaN,NaN,None,NaN,NaN,None
1,4db8d8ae8fd2b85bb2fc033953ecfc3bffa8e598,6345c275c3196199439cb92ea044ea2b53d09bdf0678da...,2014-08-04 10:00:00+00:00,2014-08-04 10:00:00+00:00,0,0.0,NaN,NaN,NaN,NaN,...,0.0,9.25,Credit Card,T.A.S. - Payment Only,NaN,NaN,None,NaN,NaN,None
2,fa49c910b6b086f773adcc31f9a2984fe669671f,8e00dda715e5ab59f03f049f4faa99fcf216fd6815d927...,2014-02-02 15:15:00+00:00,2014-02-02 15:15:00+00:00,0,0.0,NaN,NaN,NaN,NaN,...,0.0,30.78,Credit Card,Chicago Elite Cab Corp. (Chicago Carriag,NaN,NaN,None,NaN,NaN,None
3,931af0a4370dd65a98fd31cf74cf936d16adc9d3,6182ac4e117d50e00e012ec0b691c6e8fe6d22cf72a301...,2014-08-04 11:45:00+00:00,2014-08-04 11:45:00+00:00,0,0.0,NaN,NaN,NaN,NaN,...,0.0,35.10,Credit Card,Chicago Elite Cab Corp. (Chicago Carriag,NaN,NaN,None,NaN,NaN,None
4,f05c305b5dd9c0c3dc62cbcb10043623f3244a9f,be396e27bd8c66375b7a19addba6630c208b57938c5dc4...,2014-01-08 20:45:00+00:00,2014-01-08 20:45:00+00:00,360,1.3,NaN,NaN,NaN,NaN,...,0.0,6.25,Cash,Top Cab Affiliation,NaN,NaN,None,NaN,NaN,None


# Exercises

### 1) How can you predict the demand for taxis?

Say you work for a taxi company, and you're interested in predicting the demand for taxis.  Towards this goal, you'd like to create a plot that shows a rolling average of the daily number of taxi trips.  Amend the (partial) query below to return a DataFrame with two columns:
- `trip_date` - contains one entry for each date from January 1, 2016, to December 31, 2017.
- `avg_num_trips` - shows the average number of daily trips, calculated over a window including the value for the current date, along with the values for the preceding 15 days and the following 15 days, as long as the days fit within the two-year time frame.  For instance, when calculating the value in this column for January 5, 2016, the window will include the number of trips for the preceding 4 days, the current date, and the following 15 days.

This query is partially completed for you, and you need only write the part that calculates the `avg_num_trips` column.  Note that this query uses a common table expression (CTE); if you need to review how to use CTEs, you're encouraged to check out [this tutorial](https://www.kaggle.com/dansbecker/as-with) in the [Intro to SQL](https://www.kaggle.com/learn/intro-to-sql) micro-course.

In [3]:
# Fill in the blank below
avg_num_trips_query = """
                      WITH trips_by_day AS
                      (
                      SELECT DATE(trip_start_timestamp) AS trip_date,
                          COUNT(*) as num_trips
                      FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                      WHERE trip_start_timestamp >= '2016-01-01' AND trip_start_timestamp < '2018-01-01'
                      GROUP BY trip_date
                      ORDER BY trip_date
                      )
                      SELECT trip_date,
                          AVG(num_trips)
                          OVER (
                                ORDER BY trip_date
                                ROWS BETWEEN 4 PRECEDING AND 15 FOLLOWING
                               ) AS avg_num_trips
                      FROM trips_by_day
                      """

# Check your answer
q_1.check()

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,trip_date,avg_num_trips
0,2016-01-01,80461.937500
1,2016-01-02,80150.647059
2,2016-01-03,79419.611111
3,2016-01-04,79810.421053
4,2016-01-05,80293.900000


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [4]:
# Lines below will give you a hint or solution code
#q_1.hint()
q_1.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python

avg_num_trips_query = """
                      WITH trips_by_day AS
                      (
                      SELECT DATE(trip_start_timestamp) AS trip_date,
                          COUNT(*) as num_trips
                      FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                      WHERE trip_start_timestamp >= '2016-01-01' AND trip_start_timestamp < '2018-01-01'
                      GROUP BY trip_date
                      ORDER BY trip_date
                      )
                      SELECT trip_date,
                          AVG(num_trips) 
                          OVER (
                               ORDER BY trip_date
                               ROWS BETWEEN 15 PRECEDING AND 15 FOLLOWING
                               ) AS avg_num_trips
                      FROM trips_by_day
                      """

```

### 2) Can you separate and order trips by community area?

The query below returns a DataFrame with three columns from the table: `pickup_community_area`, `trip_start_timestamp`, and `trip_end_timestamp`.  

Amend the query to return an additional column called `trip_number` which shows the order in which the trips were taken from their respective community areas.  So, the first trip of the day originating from community area 1 should receive a value of 1; the second trip of the day from the same area should receive a value of 2.  Likewise, the first trip of the day from community area 2 should receive a value of 1, and so on.

Note that there are many numbering functions that can be used to solve this problem (depending on how you want to deal with trips that started at the same time from the same community area); to answer this question, please use the **RANK()** function.

In [10]:
# Amend the query below
trip_number_query = """
                WITH trips as 
                   ( SELECT pickup_community_area,
                        trip_start_timestamp,
                        trip_end_timestamp
                    FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                    WHERE DATE(trip_start_timestamp) = '2017-05-01'
                    )
                    SELECT *, 
                        RANK()
                        OVER(
                        PARTITION BY pickup_community_area
                        ORDER BY trip_start_timestamp
                        )
                        AS trip_number
                    FROM trips
                    """

# Check your answer
q_2.check()

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,pickup_community_area,trip_start_timestamp,trip_end_timestamp,trip_number
0,58.0,2017-05-01 02:30:00+00:00,2017-05-01 02:30:00+00:00,1
1,58.0,2017-05-01 05:15:00+00:00,2017-05-01 05:15:00+00:00,2
2,58.0,2017-05-01 09:00:00+00:00,2017-05-01 09:00:00+00:00,3
3,58.0,2017-05-01 09:15:00+00:00,2017-05-01 09:15:00+00:00,4
4,58.0,2017-05-01 09:30:00+00:00,2017-05-01 09:30:00+00:00,5


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [7]:
# Lines below will give you a hint or solution code
q_2.hint()
q_2.solution()

<IPython.core.display.Javascript object>

<span style="color:#3366cc">Hint:</span> You don't need a **GROUP BY** clause.  Use the **RANK()** function.  Your **OVER** clause should order the rows by the `trip_start_timestamp` column and break the data into partitions based on `pickup_community_area`.

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python

trip_number_query = """
                    SELECT pickup_community_area,
                        trip_start_timestamp,
                        trip_end_timestamp,
                        RANK()
                            OVER (
                                  PARTITION BY pickup_community_area
                                  ORDER BY trip_start_timestamp
                                 ) AS trip_number
                    FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                    WHERE DATE(trip_start_timestamp) = '2017-05-01' 
                    """

trip_number_result = client.query(trip_number_query).result().to_dataframe()

```

### 3) How much time elapses between trips?

The (partial) query in the code cell below shows, for each trip in the selected time frame, the corresponding `taxi_id`, `trip_start_timestamp`, and `trip_end_timestamp`. 

Your task in this exercise is to edit the query to include an additional `prev_break` column that shows the length of the break (in minutes) that the driver had before each trip started (this corresponds to the time between `trip_start_timestamp` of the current trip and `trip_end_timestamp` of the previous trip).  Partition the calculation by `taxi_id`, and order the results within each partition by `trip_start_timestamp`.

Some sample results are shown below, where all rows correspond to the same driver (or `taxi_id`).  Take the time now to make sure that the values in the `prev_break` column make sense to you!

![first_commands](https://i.imgur.com/qjvQzg8.png)

Note that the first trip of the day for each driver should have a value of **NaN** (not a number) in the `prev_break` column.

In [12]:
# Fill in the blanks below
break_time_query = """
                   SELECT taxi_id,
                       trip_start_timestamp,
                       trip_end_timestamp,
                       TIMESTAMP_DIFF(
                           trip_start_timestamp, 
                           LAG(trip_end_timestamp, 1)
                               OVER (PARTITION BY taxi_id ORDER BY trip_start_timestamp), 
                           MINUTE) as prev_break
                   FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                   WHERE DATE(trip_start_timestamp) = '2017-05-01' 
                   """

# Check your answer
q_3.check()

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,taxi_id,trip_start_timestamp,trip_end_timestamp,prev_break
0,12ba5aa2334dff3acea9682a3151748c671de6f91c3e35...,2017-05-01 23:45:00+00:00,2017-05-02 00:00:00+00:00,45.0
1,21339a4e7e8e216b29dd5f847f6b11bc37ef88c5ffb0d5...,2017-05-01 00:00:00+00:00,2017-05-01 00:00:00+00:00,NaN
2,21339a4e7e8e216b29dd5f847f6b11bc37ef88c5ffb0d5...,2017-05-01 00:15:00+00:00,2017-05-01 00:30:00+00:00,15.0
3,21339a4e7e8e216b29dd5f847f6b11bc37ef88c5ffb0d5...,2017-05-01 01:00:00+00:00,2017-05-01 01:00:00+00:00,30.0
4,2c125f03e3308aebd381fe53ddacf94af981e262ae3625...,2017-05-01 00:00:00+00:00,2017-05-01 00:15:00+00:00,NaN


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [11]:
# Lines below will give you a hint or solution code
#q_3.hint()
q_3.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python

break_time_query = """
                   SELECT taxi_id,
                       trip_start_timestamp,
                       trip_end_timestamp,
                       TIMESTAMP_DIFF(
                           trip_start_timestamp, 
                           LAG(trip_end_timestamp, 1) OVER (PARTITION BY taxi_id ORDER BY trip_start_timestamp), 
                           MINUTE) as prev_break
                   FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                   WHERE DATE(trip_start_timestamp) = '2017-05-01' 
                   """

break_time_result = client.query(break_time_query).result().to_dataframe()

```

# Keep going

Move on to learn how to query **[nested and repeated data](https://www.kaggle.com/alexisbcook/nested-and-repeated-data)**.

---




*Have questions or comments? Visit the [Learn Discussion forum](https://www.kaggle.com/learn-forum/161315) to chat with other Learners.*